## Phase 2: Model Conversion

This notebook converts the custom trained .h5 models into edge-optimized formats.

*   TensorFlow.js format (.json + .bin) for use in a Progressive Web App (PWA)
*   TensorFlow Lite format (.tflite) for use in mobile apps

This ensures models are ready for execution on-device, enabling real-time predictions without server calls.


**1. Setting up the environment**

In [ ]:
!pip install tensorflowjs==4.5.0 keras==2.15.0 tensorflow==2.15.0 --no-deps
!pip install tensorflow-decision-forests==1.8.1
!pip install jax==0.4.20 jaxlib==0.4.20

In [1]:
import numpy as np
import tensorflow as tf
import keras

print("✅ NumPy:", np.__version__)
print("✅ TensorFlow:", tf.__version__)
print("✅ Keras:", keras.__version__)

✅ NumPy: 1.26.4
✅ TensorFlow: 2.15.0
✅ Keras: 2.15.0


In [2]:
import os
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
import numpy as np
import pandas as pd
import shutil
import json
from datetime import datetime

In [3]:
# define paths
from google.colab import drive
drive.mount('/content/drive')

# Base paths
BASE_PATH = "/content/drive/My Drive/Edge_AI_Project/models"
H5_PATH = os.path.join(BASE_PATH, "saved_models")
TFJS_PATH = os.path.join(BASE_PATH, "tfjs_models")
TFLITE_PATH = os.path.join(BASE_PATH, "tflite_models")
LOG_PATH = os.path.join(BASE_PATH, "conversion_logs")

# Create fresh directories
os.makedirs(TFJS_PATH, exist_ok=True)
os.makedirs(TFLITE_PATH, exist_ok=True)
os.makedirs(LOG_PATH, exist_ok=True)

print("✅ Directory structure created")

Mounted at /content/drive
✅ Directory structure created


**2. Model configuration**

In [4]:
MODELS_CONFIG = {
    "age": {
        "h5_filename": "age_model.h5",
        "input_shape": (224, 224, 3),
        "description": "Adult vs Elderly classification"
    },
    "gender": {
        "h5_filename": "gender_model.h5",
        "input_shape": (224, 224, 3),
        "description": "Male vs Female classification"
    },
    "emotion": {
        "h5_filename": "emotion_model.h5",
        "input_shape": (48, 48, 1),
        "description": "Happy vs Sad classification"
    }
}

print("✅ Model configurations defined")

✅ Model configurations defined


**2. Main conversion function**

In [5]:
def convert_model_native(model_name, config):
    """
    Direct conversion using TensorFlow's native tools
    """
    print(f"\n{'='*70}")
    print(f"CONVERTING {model_name.upper()} MODEL (NATIVE APPROACH)")
    print(f"{config['description']}")
    print(f"{'='*70}")

    h5_path = os.path.join(H5_PATH, config['h5_filename'])
    tfjs_dir = os.path.join(TFJS_PATH, model_name)
    tflite_path = os.path.join(TFLITE_PATH, f"{model_name}.tflite")

    # Check if original model exists
    if not os.path.exists(h5_path):
        print(f"❌ Original model not found: {h5_path}")
        return None

    try:
        # Step 1: Load original model
        print(f" Loading original model...")
        original_model = load_model(h5_path)
        original_model = rebuild_model_with_input_shape(original_model, config['input_shape'])
        print(f"   Original: {original_model.input_shape} -> {original_model.output_shape}")
        print(f"   Layers: {len(original_model.layers)}")

        # Step 2: Test the original model (verify it works)
        print(f"Testing original model...")
        test_input = np.random.random((1,) + config['input_shape']).astype(np.float32)
        test_output = original_model.predict(test_input, verbose=0)
        print(f"   Test: {test_input.shape} -> {test_output.shape}")
        print(f"   Output range: [{test_output.min():.3f}, {test_output.max():.3f}]")

        # Step 3: Convert to TensorFlow.js (preserves all weights)
        print(f"Converting to TensorFlow.js (native)...")
        tfjs_success = convert_to_tensorflowjs_native(original_model, tfjs_dir, model_name)

        # Step 4: Convert to TensorFlow Lite (preserves all weights)
        print(f"Converting to TensorFlow Lite (native)...")
        tflite_success = convert_to_tflite_native(original_model, tflite_path, model_name)

        # Compile results
        results = {
            'model': model_name,
            'description': config['description'],
            'input_shape': config['input_shape'],
            'original_layers': len(original_model.layers),
            'clean_layers': len(original_model.layers),  # Same as original
            'weights_transferred': True,  # using original
            'tfjs_success': tfjs_success,
            'tflite_success': tflite_success,
            'tfjs_path': tfjs_dir if tfjs_success else None,
            'tflite_path': tflite_path if tflite_success else None
        }

        if tfjs_success and tflite_success:
            print(f"✅ {model_name.upper()} CONVERSION COMPLETED SUCCESSFULLY")
        else:
            print(f"⚠️  {model_name.upper()} CONVERSION COMPLETED WITH ISSUES")

        return results

    except Exception as e:
        print(f"❌ CONVERSION FAILED: {e}")
        import traceback
        traceback.print_exc()
        return None

**3. Rebuild model with input shape**

In [6]:
def rebuild_model_with_input_shape(model, input_shape):
    """
    Wraps the existing model with an Input layer to explicitly define input shape.
    This ensures TF.js conversion doesn't fail due to missing batchInputShape.
    """
    print(f"🔧 Rebuilding model with explicit Input(shape={input_shape})")
    new_input = Input(shape=input_shape)
    new_output = model(new_input)
    new_model = Model(inputs=new_input, outputs=new_output)
    return new_model


**3. TensorFlow.js conversion function**

In [7]:
def convert_to_tensorflowjs_native(model, output_dir, model_name):
    """
    Convert original Keras model directly to TensorFlow.js using native tools
    """
    print(f"   🔄 Converting {model_name} to TensorFlow.js (native)...")

    try:
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Direct conversion using tensorflowjs - this preserves the exact architecture
        tfjs.converters.save_keras_model(
            model,
            output_dir
        )

        # Check file sizes and verify conversion success
        model_json_path = os.path.join(output_dir, 'model.json')
        if os.path.exists(model_json_path):
            json_size = os.path.getsize(model_json_path) / 1024
            print(f"   📊 Model JSON: {json_size:.1f} KB")

            # Check for weight files (.bin files)
            weight_files = [f for f in os.listdir(output_dir) if f.endswith('.bin')]
            total_weight_size = sum(os.path.getsize(os.path.join(output_dir, f)) for f in weight_files)
            print(f"   📊 Model weights: {total_weight_size/1024:.1f} KB ({len(weight_files)} files)")

            print(f"   ✅ TensorFlow.js conversion successful")
            return True
        else:
            print(f"   ❌ TensorFlow.js conversion failed: model.json not found")
            return False

    except Exception as e:
        print(f"   ❌ TensorFlow.js conversion failed: {e}")
        return False

**4. TensorFlow Lite conversion function**

In [8]:
def convert_to_tflite_native(model, output_path, model_name):
    """
    Convert original Keras model directly to TensorFlow Lite using native tools
    """
    print(f"   🔄 Converting {model_name} to TensorFlow Lite (native)...")

    try:
        # Create TFLite converter from the original Keras model (preserves architecture)
        converter = tf.lite.TFLiteConverter.from_keras_model(model)

        # Apply basic optimizations for edge deployment
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        # Set quantization settings for better compatibility and smaller size
        converter.target_spec.supported_types = [tf.float16]  # Use float16 for smaller size

        # Convert the model
        tflite_model = converter.convert()

        # Save the model
        with open(output_path, 'wb') as f:
            f.write(tflite_model)

        # Check file size
        file_size = os.path.getsize(output_path) / 1024
        print(f"   📊 TFLite model: {file_size:.1f} KB")
        print(f"   ✅ TensorFlow Lite conversion successful")

        return True

    except Exception as e:
        print(f"   ❌ TensorFlow Lite conversion failed: {e}")
        return False


**5. Execution function**

In [9]:
def run_native_conversion():
    """
    Execute native conversion for all models
    """
    print(f"\nSTARTING NATIVE MODEL CONVERSION PROCESS")
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Using TensorFlow's native conversion tools")
    print(f"Preserves original model architectures and trained weights")
    print(f"="*80)

    conversion_results = []

    for model_name, config in MODELS_CONFIG.items():
        result = convert_model_native(model_name, config)
        if result:
            conversion_results.append(result)

    # Generate summary report
    print(f"\n{'='*80}")
    print(f"CONVERSION SUMMARY REPORT")
    print(f"{'='*80}")

    success_count = sum(1 for r in conversion_results if r['tfjs_success'] and r['tflite_success'])
    print(f"✅ SUCCESS STATISTICS:")
    print(f"   TensorFlow.js: {sum(1 for r in conversion_results if r['tfjs_success'])}/{len(conversion_results)} models")
    print(f"   TensorFlow Lite: {sum(1 for r in conversion_results if r['tflite_success'])}/{len(conversion_results)} models")
    print(f"")
    print(f"   ALL MODELS SUCCESSFULLY CONVERTED!")

    print(f"\n📂 DOWNLOAD LOCATIONS:")
    for result in conversion_results:
        if result['tfjs_success']:
            print(f"   {result['model']}: {result['tfjs_path']}")
        if result['tflite_success']:
            print(f"   {result['model']}: {result['tflite_path']}")

    print(f"\n✅ NATIVE CONVERSION PROCESS COMPLETED")
    print(f"="*80)

    return conversion_results

**6. Execute the conversion**

In [10]:
conversion_results = run_native_conversion()


STARTING NATIVE MODEL CONVERSION PROCESS
2025-08-04 03:37:33
Using TensorFlow's native conversion tools
Preserves original model architectures and trained weights

CONVERTING AGE MODEL (NATIVE APPROACH)
Adult vs Elderly classification
 Loading original model...
🔧 Rebuilding model with explicit Input(shape=(224, 224, 3))
   Original: (None, 224, 224, 3) -> (None, 1)
   Layers: 2
Testing original model...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


   Test: (1, 224, 224, 3) -> (1, 1)
   Output range: [0.628, 0.628]
Converting to TensorFlow.js (native)...
   🔄 Converting age to TensorFlow.js (native)...
   📊 Model JSON: 117.4 KB
   📊 Model weights: 8825.3 KB (3 files)
   ✅ TensorFlow.js conversion successful
Converting to TensorFlow Lite (native)...
   🔄 Converting age to TensorFlow Lite (native)...
   📊 TFLite model: 4360.2 KB
   ✅ TensorFlow Lite conversion successful
✅ AGE CONVERSION COMPLETED SUCCESSFULLY

CONVERTING GENDER MODEL (NATIVE APPROACH)
Male vs Female classification
 Loading original model...
🔧 Rebuilding model with explicit Input(shape=(224, 224, 3))
   Original: (None, 224, 224, 3) -> (None, 1)
   Layers: 2
Testing original model...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


   Test: (1, 224, 224, 3) -> (1, 1)
   Output range: [0.962, 0.962]
Converting to TensorFlow.js (native)...
   🔄 Converting gender to TensorFlow.js (native)...
   📊 Model JSON: 117.4 KB
   📊 Model weights: 8825.3 KB (3 files)
   ✅ TensorFlow.js conversion successful
Converting to TensorFlow Lite (native)...
   🔄 Converting gender to TensorFlow Lite (native)...
   📊 TFLite model: 4361.2 KB
   ✅ TensorFlow Lite conversion successful
✅ GENDER CONVERSION COMPLETED SUCCESSFULLY

CONVERTING EMOTION MODEL (NATIVE APPROACH)
Happy vs Sad classification
 Loading original model...
🔧 Rebuilding model with explicit Input(shape=(48, 48, 1))
   Original: (None, 48, 48, 1) -> (None, 1)
   Layers: 2
Testing original model...


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


   Test: (1, 48, 48, 1) -> (1, 1)
   Output range: [1.000, 1.000]
Converting to TensorFlow.js (native)...
   🔄 Converting emotion to TensorFlow.js (native)...
   📊 Model JSON: 7.0 KB
   📊 Model weights: 8539.5 KB (3 files)
   ✅ TensorFlow.js conversion successful
Converting to TensorFlow Lite (native)...
   🔄 Converting emotion to TensorFlow Lite (native)...
   📊 TFLite model: 4275.5 KB
   ✅ TensorFlow Lite conversion successful
✅ EMOTION CONVERSION COMPLETED SUCCESSFULLY

CONVERSION SUMMARY REPORT
✅ SUCCESS STATISTICS:
   TensorFlow.js: 3/3 models
   TensorFlow Lite: 3/3 models

   ALL MODELS SUCCESSFULLY CONVERTED!

📂 DOWNLOAD LOCATIONS:
   age: /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/age
   age: /content/drive/My Drive/Edge_AI_Project/models/tflite_models/age.tflite
   gender: /content/drive/My Drive/Edge_AI_Project/models/tfjs_models/gender
   gender: /content/drive/My Drive/Edge_AI_Project/models/tflite_models/gender.tflite
   emotion: /content/drive/My Drive/Ed

**7. BatchInputShape validation**

In [13]:
# Verifying the model structure

def inspect_graph_model(model_name):
    json_path = os.path.join(TFJS_PATH, model_name, "model.json")

    if not os.path.exists(json_path):
        print(f"❌ model.json for {model_name} not found.")
        return

    with open(json_path, "r") as f:
        model_json = json.load(f)

    print(f"\n=== {model_name.upper()} MODEL STRUCTURE ===")
    print(f"Format: {model_json.get('format', 'unknown')}")
    print(f"GeneratedBy: {model_json.get('generatedBy', 'unknown')}")

    # Check if there are signature definitions (inputs/outputs)
    topology = model_json.get("modelTopology", {})
    if "signature" in topology:
        signature = topology["signature"]
        print(f"Signature inputs: {list(signature.get('inputs', {}).keys())}")
        print(f"Signature outputs: {list(signature.get('outputs', {}).keys())}")

    # Show first few keys of topology to understand structure
    print(f"Topology keys: {list(topology.keys())}")

# Inspect all models
inspect_graph_model("age")
inspect_graph_model("gender")
inspect_graph_model("emotion")


=== AGE MODEL STRUCTURE ===
Format: layers-model
GeneratedBy: keras v2.15.0
Topology keys: ['keras_version', 'backend', 'model_config']

=== GENDER MODEL STRUCTURE ===
Format: layers-model
GeneratedBy: keras v2.15.0
Topology keys: ['keras_version', 'backend', 'model_config']

=== EMOTION MODEL STRUCTURE ===
Format: layers-model
GeneratedBy: keras v2.15.0
Topology keys: ['keras_version', 'backend', 'model_config']


In [14]:
# confirm batch_input_shape is present
import os
import json

def verify_layers_model_input_shape(model_name):
    json_path = os.path.join(TFJS_PATH, model_name, "model.json")

    with open(json_path, "r") as f:
        model_json = json.load(f)

    try:
        layers = model_json["modelTopology"]["model_config"]["config"]["layers"]
        input_layer = next(l for l in layers if l["class_name"] == "InputLayer")
        shape = input_layer["config"]["batch_input_shape"]
        print(f"✅ {model_name}: batch_input_shape = {shape}")
    except Exception as e:
        print(f"❌ Could not extract batch_input_shape for {model_name}: {e}")

verify_layers_model_input_shape("age")
verify_layers_model_input_shape("gender")
verify_layers_model_input_shape("emotion")


✅ age: batch_input_shape = [None, 224, 224, 3]
✅ gender: batch_input_shape = [None, 224, 224, 3]
✅ emotion: batch_input_shape = [None, 48, 48, 1]
